In [ ]:
#imports
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import wandb
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger

from minecraft_voxel_flow.models.flow_matching import FlowMatching, FlowMatchingCFG
from minecraft_voxel_flow.data.MNIST_datamodule import MNISTDataModule
from minecraft_voxel_flow.data.moons_datamodule import MoonsDataModule
from minecraft_voxel_flow.modules.schedulers import LinearScheduler
from minecraft_voxel_flow.modules.samplers import GaussianSampler
from minecraft_voxel_flow.modules.solvers import EulerSolver
from minecraft_voxel_flow.utils import show_imgs


In [ ]:
moons_dm = MoonsDataModule()

# --- Train Flow Matching Model ---
print("Training Flow Matching Model")
flow_model_moons = MoonsNet()
flow_model = FlowMatching(
    model=flow_model_moons,
    alpha_beta_scheduler=LinearScheduler(data_dim=2),
    sampler=GaussianSampler(target_shape=(2,)),
)
flow_trainer = Trainer(
    max_epochs=7,
    accelerator="auto",
    devices="auto",
    log_every_n_steps=50,
)
flow_trainer.fit(flow_model, moons_dm)
generated_samples = flow_model.generate_samples(steps=50, labels=[0, 1])
print(generated_samples)
print("Flow Matching Model training complete.")